  
# Water Depth Estimation using Remote Sensing and DEM
## FWDET method
 
**Author:** Jin Teng (jin.teng@csiro.au)

**Compatability:** Notebook currently compatible with the NCI.

**Dependencies:** The code below requires installation of following packages:

    rasterio
    numpy
    scipy
    sklearn

## Description

This water depth estimation tool is inspired by the Floodwater Depth Estimation Tool (FwDET) https://github.com/csdms-contrib/fwdet (Cohen et al. 2017, 2019). It is developed to add value to remote sensing (RS) analysis by calculating
water depth based solely on an inundation map with an associated digital elevation model (DEM).

The main steps of the algorithm are:

    (1) extract the grid cells that lay on the outline of the inundation extent from the raster; 
    (2) extract the DEM value (elevation) for these grid cells (referred to as boundary grid cells);
    (3) interpolate the local floodwater elevation from the boundary grid cells;
    (4) calculate floodwater depth by subtracting topographic elevation from local floodwater elevation at each grid cell within the flooded domain.

The theory here is very similar to FwDET except step 4 which utilise a geospatial interpolation using sklearn's KNN machine learning algorithm instead of allocating the local floodwater elevation for each grid cell within the flooded domain from its nearest boundary grid cell. This results in a smoother water depth raster and avoids an extra step of smoothing. However, this may cause problems when using a coarse DEM with many outliers at the boundary. The KNN interpolation code is adapted from the skspatial package https://github.com/rosskush/skspatial. It can be computational expensive with large dataset. Testing using larger scale data is underway.

We have included a run-down of the tool that is demonstrated in the notebook. 

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

## Load packages

Import Python packages that are used for the analysis.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import time
import rasterio
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from scipy import ndimage
from scipy import interpolate

## Set input variables

In [ ]:
FloodExtent_tif = os.path.join('data', 'FloodExtent_modified.tif')#'WOFS_clip.tif')
DEM_tif = os.path.join('data', 'Elevation.tif')#'dem_clip.tif')
WaterDepth_tif = os.path.join('data','output_wd_idw.tif')
WOfS_nodata_value = 0
WOfS_dry_value = 2
WOfS_wet_value = 3

## Read flood extent raster

In [ ]:
start_time = time.time()
with rasterio.open(FloodExtent_tif) as src:
    flood_extent = src.read(1)
out_mask = flood_extent!=WOfS_wet_value
not_wet = np.where(flood_extent==WOfS_dry_value,1,0)
nodata = np.where(flood_extent==WOfS_nodata_value,1,0)
print("--- %s seconds ---" % (time.time() - start_time))
plt.figure(figsize=(10,8))
plt.imshow(flood_extent,cmap='Dark2')
plt.colorbar(label='Water occurance')
plt.title('Flood Extent map')

## Extract outer boundaries

In [ ]:
wet = np.where(flood_extent==WOfS_wet_value,1,0)
start_time = time.time()
structure1 = np.ones((3,3))
bufferred = ndimage.binary_dilation(wet, structure=structure1).astype(int)
border = bufferred - wet - nodata
border = np.where(border==1,1,0)
print("--- %s seconds ---" % (time.time() - start_time))
plt.figure(figsize=(10,8))
plt.imshow(border,cmap='binary')
plt.colorbar()

## Extract inner boundaries

In [ ]:
start_time = time.time()
structure1 = np.ones((3,3))
bufferred = ndimage.binary_dilation(not_wet, structure=structure1).astype(int)
border = bufferred - not_wet - nodata
border = np.where(border==1,1,0)
print("--- %s seconds ---" % (time.time() - start_time))
plt.figure(figsize=(10,8))
plt.imshow(border,cmap='binary')
plt.colorbar()

## Read DEM

In [ ]:
with rasterio.open(DEM_tif) as src_dem:
    dem = src_dem.read(1, masked = True)
plt.figure(figsize=(10,8))
plt.imshow(dem,cmap='jet')
plt.colorbar()

## Extract DEM values using the outline

**To do: make sure the flood extent image and dem have the same projection and shape to save time on warping.**

In [ ]:
start_time = time.time()
dem_extract = border * dem
dem_extract[dem_extract<0]=0
dem_extract =np.ma.masked_where(dem_extract==0.0, dem_extract)
print("--- %s seconds ---" % (time.time() - start_time))
plt.figure(figsize=(10,8))
plt.imshow(dem_extract,cmap='jet')
plt.colorbar()

# with rasterio.open(
#     os.path.join('data','floodline_dem_extract_new.tif'), 'w',
#     driver='GTiff',
#     dtype=rasterio.float64,
#     count=1,
#     nodata=0,
#     transform = src_dem.affine,
#     width=src_dem.width,
#     height=src_dem.height) as dst:
#     dst.write(dem_extract, indexes=1)

## Interpolation using KNN

In [ ]:
start_time = time.time()
array = dem_extract.data
X = []
nrow, ncol = array.shape
frow, fcol = np.where(array > 0) # find areas where values exist
for i in range(len(frow)):
    X.append([frow[i], fcol[i]])
y = array[frow, fcol]

train_X, train_y = X, y
k=5
weights='distance'
algorithm='brute'
knn = KNeighborsRegressor(n_neighbors=k, weights=weights, algorithm=algorithm, p=2)
knn.fit(train_X, train_y)

X_pred = []
for r in range(int(nrow)):
    for c in range(int(ncol)):
        X_pred.append([r, c])
y_pred = knn.predict(np.array(X_pred))
karray = np.zeros((nrow, ncol))
i = 0
for r in range(int(nrow)):
    for c in range(int(ncol)):
        karray[r, c] = y_pred[i]
        i += 1
print("--- %s seconds ---" % (time.time() - start_time))

karray =np.ma.masked_where(out_mask, karray)
fig, axes = plt.subplots(figsize=(10,8))
plt.imshow(karray, cmap='jet')
plt.colorbar()
plt.title('z value\n')
fig.tight_layout()
CS = plt.contour(karray)
plt.clabel(CS, inline=1,
           fmt='%1.1f',
           fontsize=14)
axes.set_title('KNN')

## Estimate water depth 

Substracting ground elevation from interpolated flood water surface elevation

In [ ]:
#Interpolated floodwater elevation minus DEM
start_time = time.time()
water_depth = karray - dem
water_depth[water_depth<0] = 0
water_depth =np.ma.masked_where(out_mask, water_depth)
water_depth[water_depth.mask]=np.nan
print("--- %s seconds ---" % (time.time() - start_time))
plt.figure(figsize=(10,8))
plt.imshow(water_depth, cmap='terrain', interpolation='bilinear', vmin=-1)
plt.colorbar();
plt.title('Water Depth');

## Intetpolation using linear interpolation

In [ ]:
start_time = time.time()
array = dem_extract
nrow, ncol = array.shape
x = np.arange(0, ncol)
y = np.arange(0, nrow)
# mask invalid values
array = np.ma.masked_invalid(array)
xx, yy = np.meshgrid(x, y)
# get only the valid values
x1 = xx[~array.mask]
y1 = yy[~array.mask]
newarr = array[~array.mask]
GD1 = interpolate.griddata((x1, y1), newarr.ravel(), (xx, yy), method='linear',fill_value=np.nan)
print("--- %s seconds ---" % (time.time() - start_time))

GD1 =np.ma.masked_where(out_mask, GD1)
fig, axes = plt.subplots(figsize=(10,8))
plt.imshow(GD1, cmap='jet')
plt.colorbar()
plt.title('z value\n')
fig.tight_layout()
CS = plt.contour(GD1)
plt.clabel(CS, inline=1,
           fmt='%1.1f',
           fontsize=14)
axes.set_title('Linear')

## Estimate water depth 

Substracting ground elevation from interpolated flood water surface elevation

In [ ]:
#Interpolated floodwater elevation minus DEM
start_time = time.time()
if np.any(np.isnan(GD1)):
    GD1[np.isnan(GD1)]=dem[np.isnan(GD1)]
water_depth_l = GD1 - dem
water_depth_l[water_depth_l<0] = 0
water_depth_l =np.ma.masked_where(out_mask, water_depth_l)
water_depth_l[water_depth_l.mask]=np.nan
print("--- %s seconds ---" % (time.time() - start_time))
plt.figure(figsize=(10,8))
plt.imshow(water_depth_l, cmap='terrain', interpolation='bilinear', vmin=-1)
plt.colorbar();
plt.title('Water Depth');

## Compare the two interpolation methods

In [ ]:
knn_extract = border * karray
print("KNN border difference: %s" % np.nanmax(np.abs(dem_extract.data-knn_extract.data)))

GD1_extract = border * GD1
print("Linear border difference: %s" % np.nanmax(np.abs(dem_extract.data-GD1_extract.data)))

plt.figure(figsize=(10,8))
#plt.imshow(karray-GD1,cmap='jet')
plt.imshow(np.abs(water_depth-water_depth_l),cmap='Accent')
plt.colorbar()

## Write output

In [ ]:
with rasterio.open(
    WaterDepth_tif, 'w',
    driver='GTiff',
    dtype=water_depth_l.dtype,
    count=1,
    nodata=np.nan,
    transform = src_dem.transform,
    width=src_dem.width,
    height=src_dem.height) as dst:
    dst.write(water_depth_l, indexes=1)